# Trust Region Methods: Safe Policy Updates

Trust region methods solve one of the biggest problems in policy gradients: catastrophic policy updates!

## What You'll Learn

By the end of this notebook, you'll understand:
- The tightrope walker analogy: why small steps matter
- The problem with vanilla policy gradients
- KL divergence: measuring policy change
- TRPO (Trust Region Policy Optimization)
- Why PPO is TRPO's practical cousin

**Prerequisites:** Policy Gradient section (A2C/A3C)

**Time:** ~25 minutes

---
## The Big Picture: The Tightrope Walker Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE TIGHTROPE WALKER ANALOGY                          │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine learning to walk a tightrope...                      │
    │                                                                │
    │  VANILLA POLICY GRADIENT:                                     │
    │    "The crowd loved my last trick!"                          │
    │    → Takes HUGE leaps based on that feedback                 │
    │    → Sometimes leaps right off the rope! 💀                  │
    │    → Performance can COLLAPSE in one bad update              │
    │                                                                │
    │  TRUST REGION METHODS:                                        │
    │    "I'll only take small, careful steps"                     │
    │    → Limits how much the policy can change                   │
    │    → Guaranteed to stay on the rope (mostly!)               │
    │    → Steady, monotonic improvement                          │
    │                                                                │
    │  THE KEY INSIGHT:                                             │
    │    In RL, one bad policy update can DESTROY everything!      │
    │    Unlike supervised learning, you can't just recover.       │
    │    A bad policy collects bad data → worse policy → spiral!   │
    │                                                                │
    │  TRUST REGION = "Safety zone for policy changes"             │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, Ellipse, FancyArrowPatch
from scipy.stats import entropy

try:
    import gymnasium as gym
except ImportError:
    import gym

# Visualize the problem with large policy updates
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Vanilla Policy Gradient - Catastrophic Updates
ax1 = axes[0]
ax1.set_xlim(-1, 10)
ax1.set_ylim(-1, 10)
ax1.axis('off')
ax1.set_title('Vanilla Policy Gradient\n"Big steps can be dangerous!"', fontsize=14, fontweight='bold', color='#d32f2f')

# Draw policy landscape
x = np.linspace(0, 9, 100)
y = 4 + 2*np.sin(x*0.8) + 0.5*np.sin(x*2)
ax1.fill_between(x, 0, y, alpha=0.3, color='#90caf9')
ax1.plot(x, y, 'b-', linewidth=2)

# Show trajectory with big jumps
points = [(1, 5), (3, 6.5), (4.5, 2), (6, 0.5)]  # Catastrophic drop!
for i, (px, py) in enumerate(points):
    color = '#4caf50' if i < 2 else '#d32f2f'
    ax1.scatter([px], [py], s=100, c=color, zorder=5, edgecolors='black')
    if i < len(points) - 1:
        npx, npy = points[i+1]
        ax1.annotate('', xy=(npx, npy), xytext=(px, py),
                     arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

ax1.text(4.5, 1.5, 'COLLAPSE!', fontsize=12, color='#d32f2f', fontweight='bold', ha='center')
ax1.text(5, 8.5, 'Large gradient → Large step → Disaster', fontsize=10, ha='center')

# Right: Trust Region - Safe Updates
ax2 = axes[1]
ax2.set_xlim(-1, 10)
ax2.set_ylim(-1, 10)
ax2.axis('off')
ax2.set_title('Trust Region Methods\n"Small, safe steps only!"', fontsize=14, fontweight='bold', color='#388e3c')

# Same landscape
ax2.fill_between(x, 0, y, alpha=0.3, color='#c8e6c9')
ax2.plot(x, y, 'g-', linewidth=2)

# Show safe trajectory
safe_points = [(1, 5), (1.8, 5.8), (2.6, 6.3), (3.4, 6.5), (4.2, 6.2)]
for i, (px, py) in enumerate(safe_points):
    ax2.scatter([px], [py], s=100, c='#4caf50', zorder=5, edgecolors='black')
    # Draw trust region circle
    circle = Circle((px, py), 0.6, fill=False, linestyle='--', color='#81c784', linewidth=2)
    ax2.add_patch(circle)
    if i < len(safe_points) - 1:
        npx, npy = safe_points[i+1]
        ax2.annotate('', xy=(npx, npy), xytext=(px, py),
                     arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))

ax2.text(3, 8.5, 'Stay within trust region → Safe progress', fontsize=10, ha='center')
ax2.text(3, 3.5, 'Trust Region\n(max allowed change)', fontsize=9, ha='center', color='#388e3c')

plt.tight_layout()
plt.show()

print("\nWHY TRUST REGIONS?")
print("  • Policy gradients can have HIGH variance")
print("  • One bad update can destroy a good policy")
print("  • Unlike supervised learning, bad policy → bad data → worse policy")
print("  • Trust regions prevent catastrophic updates!")

---
## The Problem: Policy Collapse

```
    ┌────────────────────────────────────────────────────────────────┐
    │              THE POLICY COLLAPSE PROBLEM                       │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  SUPERVISED LEARNING:                                         │
    │    Bad update? → Training data stays the same                 │
    │    → Can recover in the next batch                           │
    │                                                                │
    │  REINFORCEMENT LEARNING:                                      │
    │    Bad update? → Policy becomes bad                          │
    │    → Bad policy collects BAD DATA                            │
    │    → Train on bad data → WORSE policy                        │
    │    → Vicious cycle! 🔄                                       │
    │                                                                │
    │  EXAMPLE (CartPole):                                          │
    │    Good policy: Balances well, sees diverse states           │
    │    Bad update: Now always falls left                         │
    │    Bad data: Only sees "falling" states                     │
    │    Learning: Gets even worse at balancing!                   │
    │                                                                │
    │  THE GOAL:                                                    │
    │    Ensure each update IMPROVES (or at least doesn't hurt)    │
    │    the policy. MONOTONIC IMPROVEMENT!                        │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Demonstrate the policy collapse problem

def simulate_training(collapse_at=None, n_epochs=20):
    """
    Simulate training with and without policy collapse.
    
    Args:
        collapse_at: Epoch where collapse happens (None = no collapse)
        n_epochs: Number of training epochs
    """
    np.random.seed(42)
    performance = [10]  # Start at performance 10
    
    for epoch in range(n_epochs):
        if collapse_at and epoch == collapse_at:
            # Catastrophic update!
            change = -15 + np.random.randn()
        elif collapse_at and epoch > collapse_at:
            # Bad policy = bad data = worse policy
            change = -0.5 + 0.3 * np.random.randn()  # Slow recovery or continued decline
        else:
            # Normal improvement
            change = 2 + np.random.randn()
        
        performance.append(max(0, performance[-1] + change))
    
    return performance

# Simulate both scenarios
stable_perf = simulate_training(collapse_at=None)
collapse_perf = simulate_training(collapse_at=8)

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(stable_perf, 'g-', linewidth=2, marker='o', label='Stable Training')
ax.plot(collapse_perf, 'r-', linewidth=2, marker='s', label='With Policy Collapse')

# Highlight collapse point
ax.axvline(x=8, color='red', linestyle='--', alpha=0.5)
ax.annotate('Collapse!', xy=(8, collapse_perf[8]), xytext=(10, 35),
            arrowprops=dict(arrowstyle='->', color='red'),
            fontsize=11, color='red', fontweight='bold')

ax.set_xlabel('Epoch', fontsize=11)
ax.set_ylabel('Performance', fontsize=11)
ax.set_title('The Danger of Large Policy Updates', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPOLICY COLLAPSE:")
print(f"  Without collapse: Final performance = {stable_perf[-1]:.1f}")
print(f"  With collapse: Final performance = {collapse_perf[-1]:.1f}")
print("\n  One bad update can undo all progress!")

---
## KL Divergence: Measuring Policy Change

```
    ┌────────────────────────────────────────────────────────────────┐
    │              KL DIVERGENCE: HOW DIFFERENT ARE TWO POLICIES?   │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  DEFINITION:                                                  │
    │    KL(π_old || π_new) = Σ π_old(a|s) × log(π_old(a|s)/π_new(a|s))│
    │                                                                │
    │  INTUITION:                                                   │
    │    • KL = 0: Policies are identical                          │
    │    • KL small: Policies are similar                          │
    │    • KL large: Policies are very different                   │
    │                                                                │
    │  EXAMPLE:                                                     │
    │    Old policy: [0.5, 0.3, 0.2] for actions [A, B, C]         │
    │    New policy: [0.6, 0.25, 0.15]                             │
    │    KL = 0.03 (small change, probably safe)                   │
    │                                                                │
    │    New policy: [0.1, 0.1, 0.8]                               │
    │    KL = 0.58 (big change, might be dangerous!)               │
    │                                                                │
    │  TRUST REGION CONSTRAINT:                                     │
    │    "Only update if KL(π_old || π_new) ≤ δ"                   │
    │    Where δ is a small threshold (e.g., 0.01)                 │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
def kl_divergence(p, q):
    """
    Compute KL divergence: KL(P || Q)
    
    Measures how different Q is from P.
    KL(P||Q) = Σ P(x) * log(P(x)/Q(x))
    """
    p = np.array(p) + 1e-10  # Avoid log(0)
    q = np.array(q) + 1e-10
    return np.sum(p * np.log(p / q))


# Demonstrate KL divergence
print("KL DIVERGENCE EXAMPLES")
print("="*60)

old_policy = [0.5, 0.3, 0.2]  # Original policy for 3 actions

examples = [
    ([0.5, 0.3, 0.2], "Identical policy"),
    ([0.52, 0.28, 0.20], "Tiny change"),
    ([0.6, 0.25, 0.15], "Small change"),
    ([0.7, 0.2, 0.1], "Medium change"),
    ([0.1, 0.1, 0.8], "Large change"),
    ([0.01, 0.01, 0.98], "Huge change"),
]

print(f"\nOld policy: {old_policy}")
print("\nNew policies and their KL divergence:")

kl_values = []
for new_policy, desc in examples:
    kl = kl_divergence(old_policy, new_policy)
    kl_values.append(kl)
    safety = "✓ Safe" if kl < 0.1 else "⚠ Caution" if kl < 0.3 else "✗ Dangerous!"
    print(f"  {new_policy} ({desc})")
    print(f"    KL = {kl:.4f} → {safety}")

print("\n" + "="*60)

In [ ]:
# Visualize KL divergence
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Policy distributions
ax1 = axes[0]
actions = ['Action A', 'Action B', 'Action C']
x = np.arange(len(actions))
width = 0.25

ax1.bar(x - width, old_policy, width, label='Old Policy', color='#64b5f6', edgecolor='black')
ax1.bar(x, [0.6, 0.25, 0.15], width, label='New (small Δ)', color='#81c784', edgecolor='black')
ax1.bar(x + width, [0.1, 0.1, 0.8], width, label='New (large Δ)', color='#ef5350', edgecolor='black')

ax1.set_ylabel('Probability', fontsize=11)
ax1.set_title('Policy Distributions', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(actions)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Right: KL divergence values
ax2 = axes[1]
labels = ['Same', 'Tiny', 'Small', 'Medium', 'Large', 'Huge']
colors = ['#4caf50' if kl < 0.1 else '#ffb74d' if kl < 0.3 else '#e53935' for kl in kl_values]

bars = ax2.bar(labels, kl_values, color=colors, edgecolor='black')
ax2.axhline(y=0.01, color='green', linestyle='--', linewidth=2, label='TRPO constraint (δ=0.01)')
ax2.axhline(y=0.1, color='orange', linestyle='--', linewidth=2, label='Caution zone')

ax2.set_ylabel('KL Divergence', fontsize=11)
ax2.set_title('KL Divergence: Measuring Policy Change', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nTRUST REGION CONSTRAINT:")
print("  TRPO constrains: KL(π_old || π_new) ≤ δ")
print("  Typical δ = 0.01 (very conservative)")
print("  This ensures small, safe policy updates!")

---
## TRPO: Trust Region Policy Optimization

```
    ┌────────────────────────────────────────────────────────────────┐
    │              TRPO: THE MATH                                    │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  OBJECTIVE (What we want to maximize):                        │
    │                                                                │
    │    L(θ) = E[ π_new(a|s) / π_old(a|s) × A(s,a) ]              │
    │                └────────────────────┘                         │
    │               "Probability ratio"                             │
    │                                                                │
    │    This is the "surrogate" objective                         │
    │    If ratio > 1: New policy takes this action MORE           │
    │    If ratio < 1: New policy takes this action LESS           │
    │                                                                │
    │  CONSTRAINT (Stay in trust region):                           │
    │                                                                │
    │    KL(π_old || π_new) ≤ δ                                    │
    │                                                                │
    │  TRPO SOLVES:                                                 │
    │    Maximize L(θ) subject to KL ≤ δ                           │
    │                                                                │
    │    Uses: Natural gradients + conjugate gradient + line search│
    │                                                                │
    │  THE GUARANTEE:                                               │
    │    If KL is bounded, performance is guaranteed to improve!   │
    │    (Monotonic improvement theorem)                           │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize the TRPO optimization

fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)

# Create contour plot for objective function
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
# Surrogate objective: higher in one direction
Z = -0.5*(X-1)**2 - 0.3*(Y-1)**2 + 2

contour = ax.contourf(X, Y, Z, levels=20, cmap='RdYlGn', alpha=0.7)
plt.colorbar(contour, ax=ax, label='Surrogate Objective L(θ)')

# Current policy (center)
ax.scatter([0], [0], s=200, c='blue', zorder=5, edgecolors='black', label='Current policy θ_old')

# Trust region
trust_circle = Circle((0, 0), 1.0, fill=False, linestyle='--', 
                       color='blue', linewidth=3, label='Trust region (KL ≤ δ)')
ax.add_patch(trust_circle)

# Optimal without constraint (outside trust region)
ax.scatter([1], [1], s=200, c='red', marker='x', zorder=5, linewidths=3,
           label='Unconstrained optimum (risky!)')

# TRPO solution (on trust region boundary)
# Project (1,1) onto circle of radius 1
trpo_x, trpo_y = 1/np.sqrt(2), 1/np.sqrt(2)
ax.scatter([trpo_x], [trpo_y], s=200, c='green', zorder=5, edgecolors='black',
           label='TRPO solution (safe!)')

# Arrow from old to TRPO
ax.annotate('', xy=(trpo_x, trpo_y), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', lw=2, color='green'))

# Arrow to unconstrained (crossed out)
ax.annotate('', xy=(1, 1), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', lw=2, color='red', linestyle='--'))

ax.set_xlabel('Policy Parameter θ₁', fontsize=11)
ax.set_ylabel('Policy Parameter θ₂', fontsize=11)
ax.set_title('TRPO: Maximize Objective Within Trust Region', fontsize=14, fontweight='bold')
ax.legend(loc='lower left')
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nTRPO SUMMARY:")
print("  1. Find direction that improves surrogate objective")
print("  2. But only step as far as trust region allows")
print("  3. Guaranteed improvement (or at least no catastrophe)!")

In [ ]:
def compute_surrogate_objective(old_probs, new_probs, advantages):
    """
    Compute the TRPO/PPO surrogate objective.
    
    L(θ) = E[ π_new(a|s) / π_old(a|s) × A(s,a) ]
    
    This tells us how much better the new policy is.
    """
    # Probability ratio
    ratio = new_probs / (old_probs + 1e-10)
    
    # Surrogate objective
    surrogate = ratio * advantages
    
    return surrogate.mean()


# Demonstrate surrogate objective
print("SURROGATE OBJECTIVE DEMONSTRATION")
print("="*60)

# Example: 5 state-action pairs
old_probs = np.array([0.5, 0.3, 0.4, 0.6, 0.2])  # π_old(a|s)
advantages = np.array([2.0, -1.0, 1.5, -0.5, 3.0])  # A(s,a)

print(f"\nOld policy probabilities: {old_probs}")
print(f"Advantages: {advantages}")

# Try different new policies
scenarios = [
    (old_probs, "Same policy (no change)"),
    (old_probs * 1.2, "Increase all probabilities"),  # Note: not normalized, just for demo
    (np.array([0.7, 0.2, 0.5, 0.5, 0.4]), "Increase good, decrease bad"),
]

print("\nNew policies and their surrogate objectives:")
for new_probs, desc in scenarios:
    surr = compute_surrogate_objective(old_probs, new_probs, advantages)
    ratio = new_probs / old_probs
    kl = kl_divergence(old_probs/old_probs.sum(), new_probs/new_probs.sum())
    print(f"\n  {desc}")
    print(f"    Probability ratios: {ratio.round(2)}")
    print(f"    Surrogate objective L(θ): {surr:.3f}")

print("\n" + "="*60)
print("INSIGHT: Good policy changes increase probability of high-advantage actions!")

---
## Why TRPO is Complex (and Why PPO Exists)

```
    ┌────────────────────────────────────────────────────────────────┐
    │              TRPO: POWERFUL BUT COMPLEX                        │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  TRPO REQUIRES:                                               │
    │                                                                │
    │  1. Fisher Information Matrix (FIM)                          │
    │     - Second-order information about policy                  │
    │     - Expensive to compute!                                  │
    │                                                                │
    │  2. Conjugate Gradient                                        │
    │     - Iterative solver for natural gradient                  │
    │     - Complex to implement                                   │
    │                                                                │
    │  3. Line Search                                               │
    │     - Find step size that satisfies constraint               │
    │     - Requires multiple forward passes                       │
    │                                                                │
    │  PPO'S INSIGHT:                                               │
    │    "What if we just CLIP the ratio instead?"                 │
    │    • No Fisher matrix                                        │
    │    • No conjugate gradient                                   │
    │    • No line search                                          │
    │    • Almost as good in practice!                             │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Compare TRPO vs PPO complexity

fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Left: TRPO Pipeline
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 12)
ax1.axis('off')
ax1.set_title('TRPO Pipeline\n(Complex!)', fontsize=14, fontweight='bold', color='#d32f2f')

trpo_steps = [
    ('Collect Data', '#bbdefb'),
    ('Compute Advantages', '#bbdefb'),
    ('Compute Policy Gradient', '#fff3e0'),
    ('Compute Fisher Matrix', '#ffcdd2'),
    ('Conjugate Gradient\n(natural gradient)', '#ffcdd2'),
    ('Line Search\n(find valid step)', '#ffcdd2'),
    ('Update Policy', '#c8e6c9'),
]

for i, (step, color) in enumerate(trpo_steps):
    y = 10.5 - i * 1.4
    box = FancyBboxPatch((1, y), 8, 1, boxstyle="round,pad=0.1",
                          facecolor=color, edgecolor='#333', linewidth=2)
    ax1.add_patch(box)
    ax1.text(5, y + 0.5, step, ha='center', va='center', fontsize=10)
    if i < len(trpo_steps) - 1:
        ax1.annotate('', xy=(5, y - 0.1), xytext=(5, y),
                     arrowprops=dict(arrowstyle='->', lw=1, color='#666'))

ax1.text(5, 0.5, '❌ Hard to implement\n❌ Slow per update', ha='center', fontsize=10, color='#d32f2f')

# Right: PPO Pipeline
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 12)
ax2.axis('off')
ax2.set_title('PPO Pipeline\n(Simple!)', fontsize=14, fontweight='bold', color='#388e3c')

ppo_steps = [
    ('Collect Data', '#bbdefb'),
    ('Compute Advantages', '#bbdefb'),
    ('Compute Clipped Objective', '#c8e6c9'),
    ('Standard Gradient Descent', '#c8e6c9'),
    ('Update Policy', '#c8e6c9'),
]

for i, (step, color) in enumerate(ppo_steps):
    y = 10.5 - i * 1.8
    box = FancyBboxPatch((1, y), 8, 1.2, boxstyle="round,pad=0.1",
                          facecolor=color, edgecolor='#333', linewidth=2)
    ax2.add_patch(box)
    ax2.text(5, y + 0.6, step, ha='center', va='center', fontsize=11)
    if i < len(ppo_steps) - 1:
        ax2.annotate('', xy=(5, y - 0.2), xytext=(5, y),
                     arrowprops=dict(arrowstyle='->', lw=1, color='#666'))

ax2.text(5, 1.5, '✓ Easy to implement\n✓ Fast updates\n✓ Works just as well!', 
         ha='center', fontsize=10, color='#388e3c')

plt.tight_layout()
plt.show()

print("\nPPO'S KEY INNOVATION:")
print("  Replace KL constraint with clipped objective")
print("  Same effect (bounded updates) but much simpler!")
print("  See next notebook for PPO details!")

---
## Preview: The PPO Clip Trick

```
    ┌────────────────────────────────────────────────────────────────┐
    │              PPO'S CLIPPED OBJECTIVE                           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  TRPO: Constrained optimization (complex)                     │
    │    max L(θ) such that KL ≤ δ                                  │
    │                                                                │
    │  PPO: Clip the ratio instead (simple!)                        │
    │                                                                │
    │    r(θ) = π_new(a|s) / π_old(a|s)                            │
    │                                                                │
    │    L_CLIP(θ) = min(                                          │
    │        r(θ) × A,                                             │
    │        clip(r(θ), 1-ε, 1+ε) × A                              │
    │    )                                                          │
    │                                                                │
    │  WHERE ε ≈ 0.2                                                │
    │    • Ratio can only be between 0.8 and 1.2                   │
    │    • Prevents large policy changes                           │
    │    • No constraint optimization needed!                      │
    │                                                                │
    │  THE GENIUS:                                                  │
    │    - Positive advantage + ratio > 1.2 → Clipped (no benefit) │
    │    - Negative advantage + ratio < 0.8 → Clipped (no benefit) │
    │    - No incentive to make BIG changes!                       │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize PPO clipping

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

epsilon = 0.2
ratio = np.linspace(0.5, 1.5, 100)

# Left: Positive advantage (A > 0)
ax1 = axes[0]
A = 1.0  # Positive advantage

# Unclipped objective
unclipped = ratio * A
# Clipped objective
clipped_ratio = np.clip(ratio, 1 - epsilon, 1 + epsilon)
clipped = clipped_ratio * A
# PPO objective: min of both
ppo_obj = np.minimum(unclipped, clipped)

ax1.plot(ratio, unclipped, 'b--', linewidth=2, label='Unclipped: r × A', alpha=0.7)
ax1.plot(ratio, clipped, 'r--', linewidth=2, label='Clipped: clip(r) × A', alpha=0.7)
ax1.plot(ratio, ppo_obj, 'g-', linewidth=3, label='PPO: min(both)')

ax1.axvline(x=1-epsilon, color='gray', linestyle=':', alpha=0.5)
ax1.axvline(x=1+epsilon, color='gray', linestyle=':', alpha=0.5)
ax1.axvline(x=1, color='black', linestyle='-', alpha=0.3)

ax1.fill_between(ratio, 0, ppo_obj, where=(ratio > 1 + epsilon), alpha=0.2, color='red')
ax1.text(1.35, 1.1, 'No extra\nbenefit here!', fontsize=9, ha='center', color='red')

ax1.set_xlabel('Probability Ratio r(θ)', fontsize=11)
ax1.set_ylabel('Objective', fontsize=11)
ax1.set_title('Positive Advantage (A > 0)\n"Good action - want to increase probability"', fontsize=12, fontweight='bold')
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Right: Negative advantage (A < 0)
ax2 = axes[1]
A = -1.0  # Negative advantage

# Unclipped objective
unclipped = ratio * A
# Clipped objective
clipped = clipped_ratio * A
# PPO objective: min of both (but we want to maximize, so it's max of negatives)
ppo_obj = np.minimum(unclipped, clipped)

ax2.plot(ratio, unclipped, 'b--', linewidth=2, label='Unclipped: r × A', alpha=0.7)
ax2.plot(ratio, clipped, 'r--', linewidth=2, label='Clipped: clip(r) × A', alpha=0.7)
ax2.plot(ratio, ppo_obj, 'g-', linewidth=3, label='PPO: min(both)')

ax2.axvline(x=1-epsilon, color='gray', linestyle=':', alpha=0.5)
ax2.axvline(x=1+epsilon, color='gray', linestyle=':', alpha=0.5)
ax2.axvline(x=1, color='black', linestyle='-', alpha=0.3)

ax2.fill_between(ratio, ppo_obj, 0, where=(ratio < 1 - epsilon), alpha=0.2, color='red')
ax2.text(0.65, -1.1, 'No extra\nbenefit here!', fontsize=9, ha='center', color='red')

ax2.set_xlabel('Probability Ratio r(θ)', fontsize=11)
ax2.set_ylabel('Objective', fontsize=11)
ax2.set_title('Negative Advantage (A < 0)\n"Bad action - want to decrease probability"', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nPPO CLIPPING INTUITION:")
print("  • Positive advantage: Cap how much you can increase probability")
print("  • Negative advantage: Cap how much you can decrease probability")
print("  • Result: Policy changes are bounded!")

---
## Summary: Key Takeaways

### The Trust Region Concept

| Concept | Description |
|---------|-------------|
| **Trust Region** | A boundary around current policy |
| **KL Divergence** | Measures how different two policies are |
| **Constraint** | KL(π_old \|\| π_new) ≤ δ |

### TRPO vs PPO

| Aspect | TRPO | PPO |
|--------|------|-----|
| **Constraint** | KL divergence | Clipped ratio |
| **Complexity** | High (2nd order) | Low (1st order) |
| **Implementation** | Hard | Easy |
| **Performance** | Excellent | Excellent |

### Why Trust Regions Matter

```
Without: Large update → Bad policy → Bad data → Worse policy → Collapse!
With:    Small update → Safe change → Good data → Better policy → Progress!
```

---
## Test Your Understanding

**1. Why are trust regions important in RL but not in supervised learning?**
<details>
<summary>Click to reveal answer</summary>
In supervised learning, the training data is fixed. A bad update just means lower accuracy on the same data - you can recover next batch.

In RL, the policy GENERATES its own data. A bad policy collects bad data, which makes training worse, which makes the policy even worse. This vicious cycle can cause complete collapse. Trust regions prevent this by ensuring small, safe updates.
</details>

**2. What does KL divergence measure?**
<details>
<summary>Click to reveal answer</summary>
KL divergence KL(P || Q) measures how different probability distribution Q is from P. In the context of RL:
- KL = 0 means the policies are identical
- Small KL means the policies are similar
- Large KL means the policies are very different

TRPO constrains KL(π_old || π_new) ≤ δ to ensure small policy changes.
</details>

**3. What is the surrogate objective in TRPO?**
<details>
<summary>Click to reveal answer</summary>
The surrogate objective is:
L(θ) = E[π_new(a|s) / π_old(a|s) × A(s,a)]

It measures how much better the new policy is by looking at:
- The probability ratio (how much more/less likely is this action?)
- The advantage (was this action good or bad?)

Maximizing this improves the policy.
</details>

**4. Why is TRPO complex to implement?**
<details>
<summary>Click to reveal answer</summary>
TRPO requires:
1. Computing the Fisher Information Matrix (expensive second-order computation)
2. Conjugate gradient solver to find the natural gradient direction
3. Line search to find a step size that satisfies the KL constraint

All of this is mathematically elegant but practically complex.
</details>

**5. How does PPO simplify TRPO?**
<details>
<summary>Click to reveal answer</summary>
PPO replaces the KL constraint with a clipped objective:
- Clip the probability ratio to [1-ε, 1+ε]
- Take the minimum of clipped and unclipped objectives

This achieves similar bounded updates without:
- Fisher matrix computation
- Conjugate gradient
- Line search

Just standard gradient descent!
</details>

---
## What's Next?

You now understand why trust regions matter. In the next notebook, we'll implement **PPO from scratch** - the most popular RL algorithm today!

**Continue to:** [Notebook 2: PPO From Scratch](02_ppo_from_scratch.ipynb)

---

*Trust Regions: "Small steps for the policy, giant leaps for performance!"*